In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

print(tf.__version__)

2024-02-20 15:09:25.254908: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-20 15:09:25.280676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 15:09:25.280698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 15:09:25.281348: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 15:09:25.286043: I tensorflow/core/platform/cpu_feature_guar

2.15.0


In [2]:
# loading dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
input_shape = (28, 28, 1)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train = x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test = x_test / 255.0

In [4]:
# one hot encoding
# y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

2024-02-20 15:09:26.423831: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-02-20 15:09:26.423854: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: jaysPC
2024-02-20 15:09:26.423858: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: jaysPC
2024-02-20 15:09:26.423959: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.154.5
2024-02-20 15:09:26.423969: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.154.5
2024-02-20 15:09:26.423971: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.154.5


obtain noisy data for which we trained the model and have the gradients stored in ckpt file


In [5]:
record_defaults = [tf.constant([], dtype=tf.float32)] * 10

y_train = tf.data.experimental.CsvDataset(
    filenames=["noisy_y_train.csv"], record_defaults=record_defaults
)

In [6]:
# Convert the CsvDataset to a list of EagerTensors
y_train = [tf.convert_to_tensor(value) for value in y_train]

redefine same model again

In [7]:
batch_size = 64
num_classes = 10
num_epochs = 30

In [8]:
optimizer = tf.keras.optimizers.SGD(name="SGD")
cce = tf.keras.losses.CategoricalCrossentropy()

In [9]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(
            32,
            (5, 5),
            padding="same",
            activation="relu",
            input_shape=input_shape,
            name="conv1_1",
        ),
        tf.keras.layers.Conv2D(
            32, (5, 5), padding="same", activation="relu", name="conv1_2"
        ),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(
            64, (3, 3), padding="same", activation="relu", name="conv2_1"
        ),
        tf.keras.layers.Conv2D(
            64, (3, 3), padding="same", activation="relu", name="conv2_2"
        ),
        tf.keras.layers.MaxPool2D(strides=(2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu", name="FC_1"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation="relu", name="FC_2"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax", name="FC_softmax"),
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.SGD(name="SGD"),
    loss="categorical_crossentropy",
    metrics=["acc"],
)

load checkpoint model

In [10]:
# Load gradients from the checkpoint file
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint.restore("epoch_layer_gradients.ckpt-1").expect_partial()

get gradients from the checkpoint model

In [11]:
x_train[0 : 0 + 64].shape

(64, 28, 28, 1)

In [12]:
# get indices with noisy data
indices = (np.loadtxt("noisy_indices.csv", delimiter=",")).astype(int)
indices = indices.tolist()

# convert eagertensor to numpy
y_train_numpy = [value.numpy() for value in y_train]

# create a sample of 1000 values using the 6k noisy examples
noisy_values_y = [y_train_numpy[i] for i in indices]
noisy_sample_y = noisy_values_y[:6000:6]

noisy_values_x = [x_train[i] for i in indices]
noisy_sample_x = noisy_values_x[:6000:6]

In [13]:
# get clean indices
total_indices = list(range(60000))
clean_indices = [value for value in total_indices if value not in indices]

# create a sample of 1000 values using the remaining 54k clean examples
clean_values_y = [y_train_numpy[i] for i in clean_indices]
clean_sample_y = clean_values_y[:54000:54]

clean_values_x = [x_train[i] for i in clean_indices]
clean_sample_x = clean_values_x[:54000:54]

In [14]:
print(noisy_values_x[0].shape)
noisy_sample = zip(noisy_sample_x, noisy_sample_y)
clean_sample = zip(clean_sample_x, clean_sample_y)

(28, 28, 1)


In [20]:
len(clean_sample_x)

1000

In [15]:
noisy_xTensors = tf.convert_to_tensor(noisy_sample_x)
clean_xTensors = tf.convert_to_tensor(clean_sample_x)

In [17]:
# change clean to noisy if needed
original_pred = model(clean_xTensors)

In [18]:
print(original_pred[0])
a = tf.constant(original_pred[0])
print(tf.argmax(a))

tf.Tensor(
[8.6029030e-07 2.5578109e-11 8.5242480e-09 2.0943707e-09 1.2271144e-07
 1.0125722e-06 9.9999797e-01 9.4209215e-12 4.6458113e-09 5.8680994e-12], shape=(10,), dtype=float32)
tf.Tensor(6, shape=(), dtype=int64)


In [22]:
# import json
# def makeNEmptyLists(n):
#     return [[] for _ in range(n)]

# def get_neurons(sample):
#     # define the layers to be used, since dropout or flatten don't have weights
#     conv_layers = [model.get_layer('conv1_1'), model.get_layer('conv1_2'), model.get_layer('conv2_1'), model.get_layer('conv2_2')] #
#     fc_layers = [model.get_layer('FC_1'), model.get_layer('FC_2'), model.get_layer('FC_softmax')]
#     datapointToNeuron = {conv_layers[0].name:makeNEmptyLists(1000), conv_layers[1].name:makeNEmptyLists(1000), conv_layers[2].name:makeNEmptyLists(1000), conv_layers[3].name:makeNEmptyLists(1000)}

#     for lIdx,layer in enumerate(conv_layers):
#         for neuron_index in range(layer.get_weights()[0].shape[-1]):
#             # get weights of layer
#             original_weights = layer.get_weights()
#             # save value temporarily of neuron to reset later
#             original_neuron_val = tf.identity(original_weights[0][:, :, :, neuron_index])
#             # set next neuron to 0
#             original_weights[0][:, :, :, neuron_index] = 0.0
#             # set updated weights back in model
#             layer.set_weights(original_weights)
#             # get prediction on this model
#             pred_0 = model(sample)
#             # pred_0 = tf.constant(pred_0)
#             for idx, p in enumerate(pred_0):
#                 #see if prediction changed
#                 original_predTmp = tf.argmax(original_pred[idx])
#                 newPredTMP = tf.argmax(p)
#                 if tf.argmax(original_pred[idx]) != tf.argmax(p):
#                     # append a tuple to the neurons list
#                     datapointToNeuron[layer.name][idx].append((neuron_index))
#             # map datapoint to neuron for which the prediction changed
#             original_weights[0][:, :, :, neuron_index] = original_neuron_val
#             layer.set_weights(original_weights)

#     with open("memorizing_neurons_conv.json", "w") as f:
#         json.dump(datapointToNeuron, f)

#     return datapointToNeuron

In [27]:
import json


def makeNEmptyLists(n):
    return [[] for _ in range(n)]


def get_neurons(sample):
    # define the layers to be used, since dropout or flatten don't have weights
    conv_layers = [
        model.get_layer("conv1_1"),
        model.get_layer("conv1_2"),
        model.get_layer("conv2_1"),
        model.get_layer("conv2_2"),
    ]  #
    fc_layers = [
        model.get_layer("FC_1"),
        model.get_layer("FC_2"),
        model.get_layer("FC_softmax"),
    ]
    datapointToNeuron = {
        fc_layers[0].name: makeNEmptyLists(1000),
        fc_layers[1].name: makeNEmptyLists(1000),
        fc_layers[2].name: makeNEmptyLists(1000),
    }

    for lIdx, layer in enumerate(fc_layers):
        for neuron_index in range(layer.get_weights()[0].shape[-1]):
            # get weights of layer
            original_weights = layer.get_weights()
            # save value temporarily of neuron to reset later
            original_neuron_val = tf.identity(original_weights[0][:, neuron_index])
            # set next neuron to 0
            original_weights[0][:, neuron_index] = 0.0
            # set updated weights back in model
            layer.set_weights(original_weights)
            # get prediction on this model
            pred_0 = model(sample)
            # pred_0 = tf.constant(pred_0)
            for idx, p in enumerate(pred_0):
                # see if prediction changed
                original_predTmp = tf.argmax(original_pred[idx])
                newPredTMP = tf.argmax(p)
                if tf.argmax(original_pred[idx]) != tf.argmax(p):
                    # append a tuple to the neurons list
                    datapointToNeuron[layer.name][idx].append((neuron_index))
            # map datapoint to neuron for which the prediction changed
            original_weights[0][:, neuron_index] = original_neuron_val
            layer.set_weights(original_weights)

    with open("memorizing_neurons_fc_layers.json", "w") as f:
        json.dump(datapointToNeuron, f)

    return datapointToNeuron

In [28]:
# Load gradients from the checkpoint file
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint.restore("epoch_layer_gradients.ckpt-1").expect_partial()
neurons = get_neurons(clean_xTensors)

In [29]:
# print(neurons[model.get_layer('conv2_1').name])

In [30]:
print(neurons[model.get_layer("FC_1").name])

[[], [], [], [], [], [], [], [17], [], [50, 87, 98], [98], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [40, 105], [], [], [75], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [0, 6, 17, 22, 30, 66, 68, 70, 87, 98], [], [], [], [], [], [], [18], [], [], [], [], [], [17, 22], [], [], [], [], [], [7, 66, 91], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [98], [], [], [], [], [], [], [], [], [], [], [], [98], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [7, 17, 27, 35, 59, 64, 76, 87, 91, 97, 99, 103, 104, 111, 113, 115, 116], [], [], [], [], [], [], [], [], [], [], [7, 21, 30, 36, 52, 56, 66, 99, 111, 120, 126], [], [], [], [], [12, 21], [], [], [], [], [], [], [83], 

In [ ]:
# # function to get neurons. returns a tuple containing index of neuron in each layer
# def get_neurons(noisy_sample):

#   #define the layers to be used, since dropout or flatten don't have weights
#   conv_layers = [model.get_layer('conv1_1'), model.get_layer('conv1_2'), model.get_layer('conv2_1'), model.get_layer('conv2_2')] #
#   fc_layers = [model.get_layer('FC_1'), model.get_layer('FC_2'), model.get_layer('FC_softmax')]

#   neurons = [] # list containing layer name and neuron index

#   for x, y in noisy_sample:
#     x = np.expand_dims(x, axis = 0)
#     pred = model(x)
#     pred = tf.constant(pred)

#     #
#     # Loop through convolutional layers
#     conv_layers = [model.get_layer('conv1_1'), model.get_layer('conv1_2'), model.get_layer('conv2_1'), model.get_layer('conv2_2')]
#     for layer in conv_layers:
#         for neuron_index in range(layer.get_weights()[0].shape[-1]):

#             checkpoint.restore('epoch_layer_gradients.ckpt-1').expect_partial()

#             original_weights = layer.get_weights() #get weights of layer
#             original_weights[0][:, :, :, neuron_index] = 0.0 #set first neuron to 0
#             layer.set_weights(original_weights) # set new weights to model


#             pred_1 = model(x) #get prediction on this model
#             pred_1 = tf.constant(pred_1)
#             if tf.argmax(pred_1[0]) != tf.argmax(pred[0]): #see if prediction changed
#               neurons.append((x, y, layer.name, neuron_index)) #if yes, append a tuple to the neurons list


#     # Loop through fully connected layers using same method
#     fc_layers = [model.get_layer('FC_1'), model.get_layer('FC_2'), model.get_layer('FC_softmax')]
#     for layer in fc_layers:
#         for neuron_index in range(layer.get_weights()[0].shape[-1]):

#             checkpoint.restore('epoch_layer_gradients.ckpt-1').expect_partial()
#             original_weights = layer.get_weights()
#             original_weights[0][:, neuron_index] = 0.0
#             layer.set_weights(original_weights)


#             pred_1 = model(x)
#             pred_1 = tf.constant(pred_1)

#             if tf.argmax(pred_1[0]) != tf.argmax(pred[0]):
#               neurons.append((x, y, layer.name, neuron_index))

#   return neurons

In [ ]:
# get_neurons(noisy_sample)

KeyboardInterrupt: 